# Project: Wrangling and Analyzing Data

In [1]:
#import libraries 
import pandas as pd
import numpy as np
import requests
import io
import ast
import re
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt

In [2]:
#set display options
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)

## Data Gathering

In [3]:
twitter_archive_enhanced = pd.read_csv('twitter-archive-enhanced.csv')
twitter_archive_enhanced.shape

(2356, 17)

In [4]:
request_1 = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
request_1.status_code

200

In [5]:
image_predictions = pd.read_csv(io.StringIO(request_1.content.decode('utf-8')), sep='\t')

# make it in tsv file called 'image_predictions'
image_predictions.to_csv('image_predictions.tsv', sep='\t', index=False)

image_predictions.shape

(2075, 12)

In [6]:
#API is not valid for free so i gonna read it with request
request_2 = requests.get('https://video.udacity-data.com/topher/2018/November/5be5fb7d_tweet-json/tweet-json.txt')
request_2.status_code

200

In [7]:
tweet_json = pd.read_json(request_2.text, lines=True)
tweet_json.shape

C:\Users\OS\AppData\Local\Temp\ipykernel_12192\4223544052.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tweet_json = pd.read_json(request_2.text, lines=True)


(2354, 31)

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### `twitter_archive_enhanced` Assesing

In [8]:
twitter_archive_enhanced.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN,NaN,NaN,NaN
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN,NaN,NaN,NaN
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,NaN,NaN,NaN,NaN
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,NaN,NaN,NaN,NaN


In [9]:
twitter_archive_enhanced[
(twitter_archive_enhanced['retweeted_status_id'].isna() == False)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Canela. She attempted some fancy porch pics. They were unsuccessful. 13/10 someone help her https://t.co/cLyzpcUcMX,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,"https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1,https://twitter.com/dog_rates/status/887473957103951883/photo/1",13,10,Canela,NaN,NaN,NaN,NaN
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,"https://twitter.com/dog_rates/status/886053434075471873,https://twitter.com/dog_rates/status/886053434075471873",12,10,NaN,NaN,NaN,NaN,NaN
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Lilly. She just parallel barked. Kindly requests a reward now. 13/10 would pet so well https://t.co/SATN4If5H5,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,"https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1,https://twitter.com/dog_rates/status/830583320585068544/photo/1",13,10,Lilly,NaN,NaN,NaN,NaN
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Emmy. She was adopted today. Massive round of pupplause for Emmy and her new family. 14/10 for all involved https://…,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,"https://twitter.com/dog_rates/status/878057613040115712/photo/1,https://twitter.com/dog_rates/status/878057613040115712/photo/1",14,10,Emmy,NaN,NaN,NaN,NaN
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Meet Shadow. In an attempt to reach maximum zooming borkdrive, he tore his ACL. Still 13/10 tho. Help him out below\n\nhttps:/…",8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitter.com/dog_rates/status/878281511006478336/photo/1",13,10,Shadow,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,746521445350707200,NaN,NaN,2016-06-25 01:52:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Shaggy. He knows exactly how to solve the puzzle but can't talk. All he wants to do is help. 10/10 great guy https:/…,6.678667e+17,4.196984e+09,2015-11-21 00:46:50 +0000,https://twitter.com/dog_rates/status/667866724293877760/photo/1,10,10,Shaggy,NaN,NaN,NaN,NaN
1043,743835915802583040,NaN,NaN,2016-06-17 16:01:16 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Extremely intelligent dog here. Has learned to walk like human. Even has his own dog. Very impressive 10/10 https://t.co/0Dv…,6.671383e+17,4.196984e+09,2015-11-19 00:32:12 +0000,https://twitter.com/dog_rates/status/667138269671505920/photo/1,10,10,NaN,NaN,NaN,NaN,NaN
1242,711998809858043904,NaN,NaN,2016-03-21 19:31:59 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @twitter: @dog_rates Awesome Tweet! 12/10. Would Retweet. #LoveTwitter https://t.co/j6FQGhxYuN

In [10]:
twitter_archive_enhanced.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [11]:
twitter_archive_enhanced[twitter_archive_enhanced['rating_denominator'] > 10] 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,NaN,NaN,NaN,NaN,NaN
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1",84,70,NaN,NaN,NaN,NaN,NaN
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,NaN,NaN,NaN,NaN,NaN
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244762497024/video/1,165,150,NaN,NaN,NaN,NaN,NaN
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,NaN,NaN,NaN,NaN,NaN
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023742988288/photo/1,204,170,this,NaN,NaN,NaN,NaN
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,NaN,NaN,NaN,NaN,NaN
1202,716439118184652801,NaN,NaN,2016-04-03 01:36:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/716439118184652801/photo/1,50,50,Bluebert,NaN,NaN,NaN,NaN
1228,713900603437621249,NaN,NaN,2016-03-27 01:29:02 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,NaN,NaN,NaN,https://twitter.com/dog_rates/status/713900603437621249/photo/1,99,90,NaN,NaN,NaN,NaN,NaN
1254,710658690886586372,NaN,NaN,2016-03-18 02:46:49 +0000,"<a href=""http://twitter.com/download/iphone"" r

In [12]:
twitter_archive_enhanced.duplicated().sum()

0

### `image_predictions` Assesing

In [13]:
image_predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [14]:
image_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [15]:
image_predictions['jpg_url'].duplicated().sum()

66

In [16]:
image_predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


### `tweet_json` Assesing

In [17]:
tweet_json.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,extended_entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,possibly_sensitive_appealable,lang,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2017-08-01 16:23:56+00:00,892420643555336193,892420643555336192,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,False,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}}}]}","{'media': [{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}}}]}","<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",NaN,NaN,NaN,NaN,None,"{'id': 4196983835, 'id_str': '4196983835', 'name': 'WeRateDogs™ (author)', 'screen_name': 'dog_rates', 'location': 'DM YOUR DOGS, WE WILL RATE', 'description': '#1 Source for Professional Dog Ratings | STORE: @ShopWeRateDogs | IG, FB & SC: WeRateDogs MOBILE APP: @GoodDogsGame | Business: dogratingtwitter@gmail.com', 'url': 'https://t.co/N7sNNHAEXS', 'entities': {'url': {'urls': [{'url': 'https://t.co/N7sNNHAEXS', 'expanded_url': 'http://weratedogs.com', 'display_url': 'weratedogs.com', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 3200889, 'friends_count': 104, 'listed_count': 2784, 'created_at': 'Sun Nov 15 21:41:29 +0000 2015', 'favourites_count': 114031, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 5288, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://ab...",NaN,NaN,None,NaN,False,8853,39467,False,False,0.0,0.0,en,NaN,NaN,NaN,NaN
1,2017-08-01 00:17:27+00:00,892177421306343426,892177421306343424,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",False,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 892177413194625024, 'id_str': '892177413194625024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'url': 'https://t.co/0Xxu71qeIV', 'display_url': 'pic.twitter.com/0Xxu71qeIV', 'expanded_url': 'https://twitter.com/dog_rates/status/892177421306343426/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 1407, 'h': 1600, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 598, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 105

In [18]:
# let's make data frame for json column entities and extended_entities
entities_df = pd.json_normalize(tweet_json['entities'])
extended_entities_df = pd.json_normalize(tweet_json['extended_entities'])

entities_df.head()

,hashtags,symbols,user_mentions,urls,media
0,[],[],[],[],"[{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}}}]"
1,[],[],[],[],"[{'id': 892177413194625024, 'id_str': '892177413194625024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'url': 'https://t.co/0Xxu71qeIV', 'display_url': 'pic.twitter.com/0Xxu71qeIV', 'expanded_url': 'https://twitter.com/dog_rates/status/892177421306343426/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 1407, 'h': 1600, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 598, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 1055, 'h': 1200, 'resize': 'fit'}}}]"
2,[],[],[],[],"[{'id': 891815175371796480, 'id_str': '891815175371796480', 'indices': [122, 145], 'media_url': 'http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'url': 'https://t.co/wUnZnhtVJB', 'display_url': 'pic.twitter.com/wUnZnhtVJB', 'expanded_url': 'https://twitter.com/dog_rates/status/891815181378084864/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 901, 'h': 1200, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 1201, 'h': 1600, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}}}]"
3,[],[],[],[],"[{'id': 891689552724799489, 'id_str': '891689552724799489', 'indices': [80, 103], 'media_url': 'http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'url': 'https://t.co/tD36da7qLQ', 'display_url': 'pic.twitter.com/tD36da7qLQ', 'expanded_url': 'https://twitter.com/dog_rates/status/891689557279858688/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 901, 'h': 1200, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 1201, 'h': 1600, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}}}]"
4,"[{'text': 'BarkWeek', 'indices': [129, 138]}]",[],[],[],"[{'id': 891327551943041024, 'id_str': '891327551943041024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', 'sizes': {'small': {'w': 680, 'h': 510, 'resize': 'fit'}, 'large': {'w': 720, 'h': 540, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 720, 'h': 540, 'resize': 'fit'}}}]"


In [19]:
type(entities_df.media)

pandas.core.series.Series

In [20]:
extended_entities_df.head()

,media
0,"[{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 540, 'h': 528, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 540, 'h': 528, 'resize': 'fit'}, 'medium': {'w': 540, 'h': 528, 'resize': 'fit'}}}]"
1,"[{'id': 892177413194625024, 'id_str': '892177413194625024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'url': 'https://t.co/0Xxu71qeIV', 'display_url': 'pic.twitter.com/0Xxu71qeIV', 'expanded_url': 'https://twitter.com/dog_rates/status/892177421306343426/photo/1', 'type': 'photo', 'sizes': {'large': {'w': 1407, 'h': 1600, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'small': {'w': 598, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 1055, 'h': 1200, 'resize': 'fit'}}}]"
2,"[{'id': 891815175371796480, 'id_str': '891815175371796480', 'indices': [122, 145], 'media_url': 'http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'url': 'https://t.co/wUnZnhtVJB', 'display_url': 'pic.twitter.com/wUnZnhtVJB', 'expanded_url': 'https://twitter.com/dog_rates/status/891815181378084864/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 901, 'h': 1200, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 1201, 'h': 1600, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}}}]"
3,"[{'id': 891689552724799489, 'id_str': '891689552724799489', 'indices': [80, 103], 'media_url': 'http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'url': 'https://t.co/tD36da7qLQ', 'display_url': 'pic.twitter.com/tD36da7qLQ', 'expanded_url': 'https://twitter.com/dog_rates/status/891689557279858688/photo/1', 'type': 'photo', 'sizes': {'medium': {'w': 901, 'h': 1200, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 1201, 'h': 1600, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}}}]"
4,"[{'id': 891327551943041024, 'id_str': '891327551943041024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', 'sizes': {'small': {'w': 680, 'h': 510, 'resize': 'fit'}, 'large': {'w': 720, 'h': 540, 'resize': 'fit'}, 'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'medium': {'w': 720, 'h': 540, 'resize': 'fit'}}}, {'id': 891327551947157504, 'id_str': '891327551947157504', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', '..."


In [21]:
# read data in json list
json_entities_media_df = pd.json_normalize(entities_df['media'])
json_extended_entities_media_df = pd.json_normalize(extended_entities_df['media'])

json_entities_media_df.head()

,0
0,"{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes.large.w': 540, 'sizes.large.h': 528, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.small.w': 540, 'sizes.small.h': 528, 'sizes.small.resize': 'fit', 'sizes.medium.w': 540, 'sizes.medium.h': 528, 'sizes.medium.resize': 'fit'}"
1,"{'id': 892177413194625024, 'id_str': '892177413194625024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'url': 'https://t.co/0Xxu71qeIV', 'display_url': 'pic.twitter.com/0Xxu71qeIV', 'expanded_url': 'https://twitter.com/dog_rates/status/892177421306343426/photo/1', 'type': 'photo', 'sizes.large.w': 1407, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.small.w': 598, 'sizes.small.h': 680, 'sizes.small.resize': 'fit', 'sizes.medium.w': 1055, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit'}"
2,"{'id': 891815175371796480, 'id_str': '891815175371796480', 'indices': [122, 145], 'media_url': 'http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'url': 'https://t.co/wUnZnhtVJB', 'display_url': 'pic.twitter.com/wUnZnhtVJB', 'expanded_url': 'https://twitter.com/dog_rates/status/891815181378084864/photo/1', 'type': 'photo', 'sizes.medium.w': 901, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.large.w': 1201, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.small.w': 510, 'sizes.small.h': 680, 'sizes.small.resize': 'fit'}"
3,"{'id': 891689552724799489, 'id_str': '891689552724799489', 'indices': [80, 103], 'media_url': 'http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'url': 'https://t.co/tD36da7qLQ', 'display_url': 'pic.twitter.com/tD36da7qLQ', 'expanded_url': 'https://twitter.com/dog_rates/status/891689557279858688/photo/1', 'type': 'photo', 'sizes.medium.w': 901, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.large.w': 1201, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.small.w': 510, 'sizes.small.h': 680, 'sizes.small.resize': 'fit'}"
4,"{'id': 891327551943041024, 'id_str': '891327551943041024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', 'sizes.small.w': 680, 'sizes.small.h': 510, 'sizes.small.resize': 'fit', 'sizes.large.w': 720, 'sizes.large.h': 540, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.medium.w': 720, 'sizes.medium.h': 540, 'sizes.medium.resize': 'fit'}"


In [22]:
json_extended_entities_media_df.head()

,0,1,2,3
0,"{'id': 892420639486877696, 'id_str': '892420639486877696', 'indices': [86, 109], 'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg', 'url': 'https://t.co/MgUWQ76dJU', 'display_url': 'pic.twitter.com/MgUWQ76dJU', 'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1', 'type': 'photo', 'sizes.large.w': 540, 'sizes.large.h': 528, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.small.w': 540, 'sizes.small.h': 528, 'sizes.small.resize': 'fit', 'sizes.medium.w': 540, 'sizes.medium.h': 528, 'sizes.medium.resize': 'fit'}",None,None,None
1,"{'id': 892177413194625024, 'id_str': '892177413194625024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg', 'url': 'https://t.co/0Xxu71qeIV', 'display_url': 'pic.twitter.com/0Xxu71qeIV', 'expanded_url': 'https://twitter.com/dog_rates/status/892177421306343426/photo/1', 'type': 'photo', 'sizes.large.w': 1407, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.small.w': 598, 'sizes.small.h': 680, 'sizes.small.resize': 'fit', 'sizes.medium.w': 1055, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit'}",None,None,None
2,"{'id': 891815175371796480, 'id_str': '891815175371796480', 'indices': [122, 145], 'media_url': 'http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg', 'url': 'https://t.co/wUnZnhtVJB', 'display_url': 'pic.twitter.com/wUnZnhtVJB', 'expanded_url': 'https://twitter.com/dog_rates/status/891815181378084864/photo/1', 'type': 'photo', 'sizes.medium.w': 901, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.large.w': 1201, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.small.w': 510, 'sizes.small.h': 680, 'sizes.small.resize': 'fit'}",None,None,None
3,"{'id': 891689552724799489, 'id_str': '891689552724799489', 'indices': [80, 103], 'media_url': 'http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg', 'url': 'https://t.co/tD36da7qLQ', 'display_url': 'pic.twitter.com/tD36da7qLQ', 'expanded_url': 'https://twitter.com/dog_rates/status/891689557279858688/photo/1', 'type': 'photo', 'sizes.medium.w': 901, 'sizes.medium.h': 1200, 'sizes.medium.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.large.w': 1201, 'sizes.large.h': 1600, 'sizes.large.resize': 'fit', 'sizes.small.w': 510, 'sizes.small.h': 680, 'sizes.small.resize': 'fit'}",None,None,None
4,"{'id': 891327551943041024, 'id_str': '891327551943041024', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', 'sizes.small.w': 680, 'sizes.small.h': 510, 'sizes.small.resize': 'fit', 'sizes.large.w': 720, 'sizes.large.h': 540, 'sizes.large.resize': 'fit', 'sizes.thumb.w': 150, 'sizes.thumb.h': 150, 'sizes.thumb.resize': 'crop', 'sizes.medium.w': 720, 'sizes.medium.h': 540, 'sizes.medium.resize': 'fit'}","{'id': 891327551947157504, 'id_str': '891327551947157504', 'indices': [139, 162], 'media_url': 'http://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg', 'media_url_https': 'https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg', 'url': 'https://t.co/AtUZn91f7f', 'display_url': 'pic.twitter.com/AtUZn91f7f', 'expanded_url': 'https://twitter.com/dog_rates/status/891327558926688256/photo/1', 'type': 'photo', 'sizes.small.w': 680, 'sizes.small.h': 510, 'sizes.small.r

In [23]:
# let's make json data frames read able
entities_media_df = pd.json_normalize(json_entities_media_df[0])
extended_entities_media_df_0 = pd.json_normalize(json_extended_entities_media_df[0])
extended_entities_media_df_1 = pd.json_normalize(json_extended_entities_media_df[1])
extended_entities_media_df_2 = pd.json_normalize(json_extended_entities_media_df[2])
extended_entities_media_df_3 = pd.json_normalize(json_extended_entities_media_df[3])

#let's concat all extended_entities_media data frames
media_df = pd.concat([entities_media_df, pd.json_normalize([df for df in [f'extended_entities_media_df_{ind}' for ind in [0, 1, 2, 3]]])])

media_df.head()

,id,id_str,indices,media_url,media_url_https,url,display_url,expanded_url,type,sizes.large.w,sizes.large.h,sizes.large.resize,sizes.thumb.w,sizes.thumb.h,sizes.thumb.resize,sizes.small.w,sizes.small.h,sizes.small.resize,sizes.medium.w,sizes.medium.h,sizes.medium.resize,source_status_id,source_status_id_str,source_user_id,source_user_id_str
0,8.924206e+17,892420639486877696,"[86, 109]",http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,https://t.co/MgUWQ76dJU,pic.twitter.com/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,photo,540.0,528.0,fit,150.0,150.0,crop,540.0,528.0,fit,540.0,528.0,fit,NaN,NaN,NaN,NaN
1,8.921774e+17,892177413194625024,"[139, 162]",http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,https://t.co/0Xxu71qeIV,pic.twitter.com/0Xxu71qeIV,https://twitter.com/dog_rates/status/892177421306343426/photo/1,photo,1407.0,1600.0,fit,150.0,150.0,crop,598.0,680.0,fit,1055.0,1200.0,fit,NaN,NaN,NaN,NaN
2,8.918152e+17,891815175371796480,"[122, 145]",http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,https://t.co/wUnZnhtVJB,pic.twitter.com/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,photo,1201.0,1600.0,fit,150.0,150.0,crop,510.0,680.0,fit,901.0,1200.0,fit,NaN,NaN,NaN,NaN
3,8.916896e+17,891689552724799489,"[80, 103]",http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,https://t.co/tD36da7qLQ,pic.twitter.com/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,photo,1201.0,1600.0,fit,150.0,150.0,crop,510.0,680.0,fit,901.0,1200.0,fit,NaN,NaN,NaN,NaN
4,8.913276e+17,891327551943041024,"[139, 162]",http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,https://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,https://t.co/AtUZn91f7f,pic.twitter.com/AtUZn91f7f,https://twitter.com/dog_rates/status/891327558926688256/photo/1,photo,720.0,540.0,fit,150.0,150.0,crop,680.0,510.0,fit,720.0,540.0,fit,NaN,NaN,NaN,NaN


In [24]:
media_df['id_str'].iloc[2],  media_df['expanded_url'].iloc[2]

('891815175371796480',
 'https://twitter.com/dog_rates/status/891815181378084864/photo/1')

In [25]:
twitter_archive_enhanced[twitter_archive_enhanced['expanded_urls'] == 'https://twitter.com/dog_rates/status/891815181378084864/photo/1']

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN,NaN,NaN,NaN


by comparing **id** in id_str column and **id** in the link of expanded_url column found that id not right but id in the url is right because it found in other data frames but id not found

In [26]:
media_df.duplicated(subset = ['id']).sum()

349

In [27]:
media_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2358 entries, 0 to 3
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2073 non-null   float64
 1   id_str                2073 non-null   object 
 2   indices               2073 non-null   object 
 3   media_url             2073 non-null   object 
 4   media_url_https       2073 non-null   object 
 5   url                   2073 non-null   object 
 6   display_url           2073 non-null   object 
 7   expanded_url          2073 non-null   object 
 8   type                  2073 non-null   object 
 9   sizes.large.w         2073 non-null   float64
 10  sizes.large.h         2073 non-null   float64
 11  sizes.large.resize    2073 non-null   object 
 12  sizes.thumb.w         2073 non-null   float64
 13  sizes.thumb.h         2073 non-null   float64
 14  sizes.thumb.resize    2073 non-null   object 
 15  sizes.small.w         2073 no

### Quality issues
1. **Accuracy issue**: to more accurate of data must delete retweets in rows 

2. **Accuracy issue**: data type of timestamp column in *twitter_archive_enhanced* data frame is object must be datetime data type into datetime data type

3. **Accuracy issue**: there is zero value and values greater than 10 in rating_denominator column at *twitter_archive_enhanced* data frame which must be 10 like others

4. **copmletness issue**: missing values in *media_df* data frame column **id**

5. **copmletness issue**: there is a duplicated columns has alot of nan values with other names in *media_df* from column with index [29] to [24] with 2280 null values

6. **uniqueness issue**: there is duplicated values in id column at *media_df* data frame

7. **uniqueness issue**: there is duplicated values in jpg_url column in data frame *image_predictions*

8. **Accuracy issue**: the rating_numerator column has values very high due to rating_denominator column is greater than 10 at *twitter_archive_enhanced* data frame which must be devided to reach ten and devided numrator with same number to make correct ratio

9. **copmletness issue**: there is columns with alot of null values and retweeted no helpful columns at *twitter_archive_enhanced* data frame

10. **copmletness issue**: there is null values in column name in *twitter_archive_enhanced* data frame

11. **Accuracy issue**: id not right in id_str column at *media_df* data frame

12. **Accuracy issue**: duplicated and untidy values in expanded_urls column at *twitter_archive_enhanced* data frame


### Tidiness issues
1. make in *twitter_archive_enhanced* data frame column for breeds (doggo, floofer, puppo, pupper)

2. id_str column is breferd to name tweet_id as other data frames to facilitate merging and analysing data at *media_df* data frame

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [28]:
# make copy for data frames to not editing in original one
twitter_archive_enhanced_cleaned = twitter_archive_enhanced.copy()
media_df_cleaned = media_df.copy()
image_predictions_cleaned = image_predictions.copy()

### Quality Issue #1
### Define:
> to more accurate of data must delete retweets in rows by function `.str.contains()` adn `.drop()`
### Coding:

In [33]:
# drop retweets in text column
twitter_archive_enhanced_cleaned = twitter_archive_enhanced_cleaned[~twitter_archive_enhanced_cleaned['text'].str.contains("RT @")]

# drop rows in retweeted_status_id column
twitter_archive_enhanced_cleaned = twitter_archive_enhanced_cleaned[pd.isnull(twitter_archive_enhanced_cleaned['retweeted_status_user_id'])]

# check code result by see how many null values in rows retweeted
twitter_archive_enhanced_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        1495 non-null   object 
 13  doggo                       87 non-nul

### Quality Issue #2
### Define:
> convert data type of **timestamp** column in *twitter_archive_enhanced data frame* with function `.to_datetime`
### Coding:

In [ ]:
twitter_archive_enhanced_cleaned['timestamp'] = pd.to_datetime(twitter_archive_enhanced_cleaned['timestamp'])
# check code result
twitter_archive_enhanced_cleaned['timestamp'].dtype

### Quality Issue #3
### Define:
> there is zero (may be there is values less than ten not knowing) and values greater than 10 value in **rating_denominator** column at *twitter_archive_enhanced* data frame which must be 10 like others solv values smaller than 10 with boolean indexing `['rating_denominator'] < 10 ` and solve values greater than 10 with `
### Coding:

In [ ]:
twitter_archive_enhanced_cleaned[twitter_archive_enhanced_cleaned['rating_denominator'] < 10] = 10
# check code result
twitter_archive_enhanced_cleaned['rating_denominator'].describe() 

### Quality Issue #4
### Define:
> there is alot missing values in *media_df* and can not fill them due to it does not has no data in some rows so will solve it with drop null values in **id** column by function `.dropna` to get the most helpfull data can clean others
### Coding:

In [ ]:
media_df_cleaned.dropna(subset = ['id'], inplace = True)
media_df_cleaned.info()

### Quality Issue #5
### Define:
> there is a duplicated columns has alot of nan values with other names in *media_df* from column with index [21] to [24] with 2280 null values solved with function `.drop`
### Coding:

In [ ]:
cols_to_drop = [ind for ind in range(21, 25)]
media_df_cleaned.drop(media_df_cleaned.columns[cols_to_drop], axis=1, inplace=True)
# check code result
media_df_cleaned.info()

### Quality Issue #6
### Define:
> there is alot duplicated values in *media_df* so will solve it with drop duplicated values in **id** column to get the most helpfull data can clean others solved with function `.drop_duplicates()`
### Coding:

In [ ]:
media_df_cleaned.drop_duplicates(subset = ['id'], inplace = True)
# check code result
media_df_cleaned['id'].duplicated().sum()

### Quality Issue #7
### Define:
> there is duplicated values in jpg_url column in data frame *image_predictions* solved with function `.drop_duplicates()`
### Coding:

In [ ]:
image_predictions_cleaned.drop_duplicates(subset = 'jpg_url', inplace= True)
# check code result
image_predictions_cleaned['jpg_url'].duplicated().sum()

### Quality Issue #8
### Define:
> **Accuracy issue**: the rating_numerator column has values very high due to rating_denominator column is greater than 10 at *twitter_archive_enhanced* data frame which must be devided to reach ten and devided numrator with same number to make correct ratio solved with making function check denominator to correct numerator solved with `lambda`
### Coding:

In [ ]:
# devide numerator has denominator larger than 10 with number devide it's denominator to be 10
twitter_archive_enhanced_cleaned['rating_numerator'][twitter_archive_enhanced_cleaned['rating_denominator'] > 10] = twitter_archive_enhanced_cleaned['rating_numerator'] / (lambda x: x // 10)(twitter_archive_enhanced_cleaned['rating_denominator'])

# twitter_archive_enhanced_cleaned.head()
twitter_archive_enhanced_cleaned[twitter_archive_enhanced_cleaned['rating_denominator'] > 10]

In [ ]:
# make all denominator greater than 10 equal 10
twitter_archive_enhanced_cleaned[twitter_archive_enhanced_cleaned['rating_denominator'] > 10] = 10

### Quality Issue #9
### Define:
> there is columns ('in_reply_to_user_id', 'in_reply_to_status_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp') with alot of null values and retweeted no helpful columns at *twitter_archive_enhanced* data frame solved with function `.drop()`
### Coding:

In [ ]:
twitter_archive_enhanced_cleaned.drop(columns=['in_reply_to_user_id', 'in_reply_to_status_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], inplace=True, errors='ignore')
# check code result
twitter_archive_enhanced_cleaned.info()

### Quality Issue #10
### Define:
> there is null values in column name in *twitter_archive_enhanced* data frame solved with function `.fillna()`
### Coding:

In [ ]:
twitter_archive_enhanced_cleaned.fillna({'name' : 'no name'}, inplace = True)
# check code result
twitter_archive_enhanced_cleaned['name'].isnull().sum()

### Quality Issue #11
### Define:
> id not right in id_str column at *media_df* data frame by comparing with id found in it's link in expanded_url column solved with make column id_str take new values from column expanded_url by making a function and apply it to expanded_url to extract id from link with regular expression
### Coding:

In [ ]:
def extract_tweet_id(url):
    match = re.search(r'/status/(\d+)', url)
    return match.group(1) if match else None
    
media_df_cleaned['id_str'] = media_df_cleaned['expanded_url'].apply(extract_tweet_id)
# check code result
media_df_cleaned['id_str'].iloc[2], media_df_cleaned['expanded_url'].iloc[2]

by comparing id in id_str column and one in the link at expanded_url column found that problem solved

### Quality Issue #12
### Define:
> there is duplicating values and not accuracy data in  *twitter_archive_enhanced* data frame **expanded_urls** column will solved
### Coding:

In [ ]:
twitter_archive_enhanced_cleaned['expanded_urls'] = 'https://twitter.com/dog_rates/status/' + twitter_archive_enhanced_cleaned['tweet_id'].astype(str) + '/photo/1'
# check code result
twitter_archive_enhanced_cleaned.sample()

### Tidiness Issue #1
### Define:
> make new column called **breed** in *twitter_archive_enhanced* data frame to classify (doggo, floofer, pupper, puppo) with making function classify them with function `.str.contains()` and `.loc`
### Coding:

In [ ]:
# find it text the breed of dog
twitter_archive_enhanced_cleaned.loc[twitter_archive_enhanced['text'].str.contains('doggo'), 'breed'] = 'doggo'
twitter_archive_enhanced_cleaned.loc[twitter_archive_enhanced['text'].str.contains('floofer'), 'breed'] = 'floofer'
twitter_archive_enhanced_cleaned.loc[twitter_archive_enhanced['text'].str.contains('pupper'), 'breed'] = 'pupper'
twitter_archive_enhanced_cleaned.loc[twitter_archive_enhanced['text'].str.contains('puppo'), 'breed'] = 'puppo'

# check for hygen breeds which have two breeds together
for index, row in twitter_archive_enhanced_cleaned.iterrows():
    tweet_id = row['tweet_id']
    doggo = row['doggo']
    floofer = row['floofer']
    pupper = row['pupper']
    puppo = row['puppo']
    
    # Check for combinations of breeds and update the 'breed' column
    hygens = {
    ('doggo', 'floofer'): 'hygen(doggo+floofer)',
    ('doggo', 'pupper'): 'hygen(doggo+pupper)',
    ('doggo', 'puppo'): 'hygen(doggo+puppo)',
    ('floofer', 'pupper'): 'hygen(floofer+pupper)',
    ('floofer', 'puppo'): 'hygen(floofer+puppo)',
    ('pupper', 'puppo'): 'hygen(pupper+puppo)'
    }

    for (a, b), breed in hygens.items():
        if not pd.isna(locals()[a]) and not pd.isna(locals()[b]):
            twitter_archive_enhanced_cleaned.loc[twitter_archive_enhanced_cleaned['tweet_id'] == tweet_id, 'breed'] = breed
            break


# drop columns (doggo, floofer, pupper, puppo)
twitter_archive_enhanced_cleaned.drop(columns= ['doggo', 'floofer', 'pupper', 'puppo'], inplace= True)

# fill na with 'no breed' value
twitter_archive_enhanced_cleaned['breed'].fillna('no breed', inplace = True)

# check code result
twitter_archive_enhanced_cleaned.sample()

In [ ]:
# hygen sample doggo + pupper
twitter_archive_enhanced_cleaned[twitter_archive_enhanced_cleaned['tweet_id'] == 781308096455073793]

In [ ]:
twitter_archive_enhanced_cleaned['breed'].value_counts()

### Tidiness Issue #2
### Define:
> id_str column is breferd to name tweet_id as other data frames to facilitate merging and analysing data at *media_df* data frame solved with function `.rename` to rename column to tweet_id
### Coding:

In [ ]:
# rename column id_str with tweet_id and convert it's type
media_df_cleaned.rename(columns = {'id_str' : 'tweet_id'}, inplace = True)

# check code result
media_df_cleaned.info()

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [ ]:
# Convert tweet_id columns to string
for df in [twitter_archive_enhanced_cleaned, media_df_cleaned, image_predictions_cleaned]:
    df['tweet_id'] = df['tweet_id'].astype(str)
    
# merge all data frames cleaned in master_df
master_df = pd.merge(twitter_archive_enhanced_cleaned, media_df_cleaned, on='tweet_id', how='inner').merge(image_predictions_cleaned, on='tweet_id', how='inner')

master_df.info()

In [ ]:
# make master_df in twitter_archive_master.csv
master_df.to_csv('twitter_archive_master.csv', index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1. pupper breed is the spreadest breed between other breeds.

2. breed puppo is the highest rated one.

3. breed floofer is the least rated one.

In [ ]:
# konowing spreadest breed
master_df['breed'].value_counts()

In [ ]:
# to know the rating of breeds
[f"sum of {breed} rating is {master_df['rating_numerator'][master_df['breed'] == breed].sum()}" for breed in ['doggo', 'pupper', 'puppo', 'floofer']]

### Visualization

In [ ]:
fig, mm = plt.subplots(figsize =(10,6))
breed_counts = master_df['breed'].value_counts()
plt.barh(breed_counts.index, breed_counts.values, color='#FF7F50')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram for breed spreadity')
plt.show()